# PART 1

Industry member with:
•         High percentage of supervisor transactions (include the average as a benchmark) 
•         High percentage of transactions with one staff member (include the average as a benchmark)
•         High percentage of transactions with one staff member at multiple times of the day 
•         High percentage of transactions with one staff member over different transaction types


In [315]:
import pandas as pd

import numpy as np

path = "C:\\Users\\dmehri\\Documents\\DATA\\AEU Network Analysis\\"

df = pd.read_csv(path + "AEU Data Sept 2017 - Sept 2018.csv")

col_names = sorted(list(df.columns.values))

#strip all columns of white space and convert to uppper
for i in range(0, len(col_names)):
    df[col_names[i]] = df[col_names[i]].astype(str)
    df[col_names[i]] = df[col_names[i]].map(str.strip)
    df[col_names[i]] = df[col_names[i]].map(str.upper)
    
df = df.rename(columns={'Service Category': 'Service_Category'})

    
#FILTER ONLY FOR AEU    
print "Length before drop", len(df)
df = df[df.Service_Category.str.contains("AEU") == True]
df = df.reset_index(drop=True)
print "lenght after drop", len(df)

print "Delivered Service "
del_service = set(df["DELIVERED_SERVICE"].tolist() )
print del_service

print "Service Catgories"
serv_cat = set(df["Service_Category"].tolist() )
print serv_cat
    
    
df["STAFF NAME"] = df["STAFF FIRST_NAME"] + " " + df["STAFF LAST_NAME"]

df = df.drop('STAFF FIRST_NAME', 1)
df = df.drop('STAFF LAST_NAME', 1)  
df = df.drop('SECONDS', 1)
df = df.drop('Staff User Name', 1)

df["CUSTOMER NAME RAW"] = df['CUSTOMER NAME']

#Clean customer name
df['CUSTOMER NAME'] = df['CUSTOMER NAME'].str.replace(',', ' ')
#df['CUSTOMER NAME'] = df['CUSTOMER NAME'].str.replace(',', ' ')
df['CUSTOMER NAME'] = df['CUSTOMER NAME'].str.replace('CUSTOMER ', '')
df['CUSTOMER NAME'] = df['CUSTOMER NAME'].str.replace('HOMEOWNER ', '')

df["CUSTOMER NAME"] = df["CUSTOMER NAME"].map(str.strip)


Length before drop 761192
lenght after drop 50915
Delivered Service 
set(['CERTIFICATE OF CORRECTION', 'DROP OFF (MULTIPLE SUBMISSION)', 'TICKET DOES NOT MATCH ID', 'CERTIFICATE OF CORRECTION (IN-PERSON)', 'INQUIRY', 'PICK-UP RESULTS', 'UNPREPARED', 'DROP OFF', 'INQUIRY, PICK UP RESULTS', 'PICK UP RESULTS', 'DROP OFF (10X10)'])
Service Catgories
set(['1ST-REP- AEU PICK-UP', '1ST-RESP- AEU GENERAL', '1ST-RESP- AEU SERVICES', 'RESP - AEU SERVICES', '1ST-REP-AEU INQUIRIES', 'REP-AEU DROP-OFF', 'RESP-AEU INQUIRIES', 'REP-AEU PICK-UP', 'REP-AEU INQUIRIES', '1ST-RESP-AEU PICK-UP', '1ST-RESP-AEU INQUIRIES', '1ST-RESP-AEU DROP-OFF', 'REP -  AEU SERVICES', '1ST-REP- AEU SERVICES', 'RESP-AEU PICK-UP', 'RESP-AEU DROP-OFF', '1ST-REP-AEU DROP-OFF'])


In [316]:
#df.to_csv(path + "testAEU.csv")

CREATE INDUSTRY NAME DATAFRAME

In [317]:
pd.set_option('chained_assignment', None) 

industry_names =  list(set(df["CUSTOMER NAME"].tolist() ) ) 
print "number of customers", len(industry_names)

#industry_names_df = pd.DataFrame(industry_names)

df_final = df[["CUSTOMER NAME"] ] 

df_final["Transaction Count"] = 1

df_final = df_final.groupby(['CUSTOMER NAME']).sum()

#Reset back to dataframe:
df_final = df_final.add_suffix('').reset_index()


#industry_names_df = industry_names_df.rename(columns={0: 'CUSTOMER NAME'})

number of customers 6704


SUPERVISOR INTERACTION
Two Supervisors: Declan Daly, Dominique Livingston

In [318]:
df2 = df.copy()

df2['Livingston Transaction'] = np.where(df2['STAFF NAME'].str.contains("DOMINIQUE LIVINGSTON", case=False, na=False), 1, 0)
df2['Daly Transaction'] = np.where(df2['STAFF NAME'].str.contains("DECLAN DALY", case=False, na=False), 1, 0)

df2 = df2[["CUSTOMER NAME", "Livingston Transaction", "Daly Transaction"]]

df2G = (df2.groupby('CUSTOMER NAME').agg({'Livingston Transaction':'sum', 'Daly Transaction': 'sum'}).reset_index().rename(columns={'Livingston Transaction':'Livingston Transaction'}) )


#MERGE WTIH INDUSTRY NAME DF
df_final = pd.merge(df_final, df2G, how='left', on=['CUSTOMER NAME'])

df_final["Total Supervisor Transaction"] = df2G["Daly Transaction"] + df2G["Livingston Transaction"]


Summary Stats

In [319]:
average_transactions = df_final["Transaction Count"].mean()

print average_transactions

median_transactions = df_final["Transaction Count"].median()

print median_transactions



7.59471957041
2.0


SUBSET FOR >= 5 TRANSACTIONS

In [320]:
df_final = df_final.sort_values(by = 'Transaction Count', ascending=False).reset_index(drop=True)

print "Length before subset", len(df_final)

df_final = df_final[df_final["Transaction Count"] >= 5]
df_final = df_final.reset_index(drop=True)

print "Length after subset", len(df_final)

Length before subset 6704
Length after subset 1447


SUPERVISOR TRANSACTION PERCENT 

In [321]:
df_final["Transaction Count"] = df_final["Transaction Count"].astype(float)
df_final["% Daly"] = ( (df_final["Daly Transaction"]/df_final["Transaction Count"]).round(3) ) *100
mean_daly = df_final["% Daly"].mean()
print mean_daly

df_final["% Livingston"] = ( (df_final["Livingston Transaction"]/df_final["Transaction Count"] ).round(3)) *100
mean_liv = df_final["% Livingston"].mean()
print mean_liv

df_final["% Daly"].describe().transpose()

5.99488597097
0.257014512785


count    1447.000000
mean        5.994886
std        10.526524
min         0.000000
25%         0.000000
50%         0.000000
75%         8.300000
max        66.700000
Name: % Daly, dtype: float64

In [322]:
std = df_final["% Daly"].describe().transpose()

std

count    1447.000000
mean        5.994886
std        10.526524
min         0.000000
25%         0.000000
50%         0.000000
75%         8.300000
max        66.700000
Name: % Daly, dtype: float64

BAD ACTOR FLAG

In [323]:
#badAllRoles.to_csv(path + "test.csv", index=False)

In [324]:
import ngram
#ngram.NGram.compare('Ham','Spam',N=1)
from datetime import datetime
print str(datetime.now())
from operator import itemgetter


badAllRoles = pd.read_csv("C:\\Users\\dmehri\\Documents\\DATA\\AEU Network Analysis\\Bad Actors for Darius AEU Transaction Analysis.csv")

print "len before drop", len(badAllRoles)

badAllRoles = badAllRoles.dropna(subset = ['First Name']).reset_index(drop=True)

print "len after drop", len(badAllRoles)

col_names = sorted(list(badAllRoles.columns.values))

for i in range(0, len(col_names)):
    badAllRoles[col_names[i]] = badAllRoles[col_names[i]].astype(str)
    badAllRoles[col_names[i]] = badAllRoles[col_names[i]].map(str.strip)
    badAllRoles[col_names[i]] = badAllRoles[col_names[i]].map(str.upper)


badAllRoles["Name"] = badAllRoles["First Name"] + " " + badAllRoles["Last Name"]

badAllRoles = badAllRoles[["Name", "Role"]]

badAllRoles = badAllRoles.sort_values(by = 'Name', ascending=True).reset_index(drop=True)

badAllRoles = badAllRoles.drop_duplicates(subset=['Name', 'Role']).reset_index(drop=True)

print "len after drop", len(badAllRoles)


#badBusiness = badBusiness["Name"].tolist()
#badPeople = badPeople["Name"].tolist()

#badActor = badBusiness + badPeople

count_bad = 0


badActor = badAllRoles["Name"].tolist()

df_final["Bad Actor Flag"] = 0

for i in range(0, len(df_final) ):
    if df_final["CUSTOMER NAME"][i] in badActor:
        df_final["Bad Actor Flag"][i] = 1
        count_bad += 1 
               
print "number of bad actors", count_bad


print "Start fuzzy matching"

#FUZZY MATCHING

#df_final["Name Match"] = ""
#df_final["Name Match Score"] = ""


def my_fuzzy(target, match):
  
    G = ngram.NGram(match)
    
    remove_list = []
        
    for i in range(0, len(target)):
    #for i in range(0, 1):
        #print "*****************************"
        tname = target["CUSTOMER NAME"][i]
        #print i
        #print tname
    
        if i%500 == 0:
            print "iteration", i
            print str(datetime.now())

      
        ratio_list =  G.search(tname,threshold=0.63)
        #ratio_list =  G.search(tname)

        
        #print ratio_list[0][1]
        
        if len(ratio_list) > 0:
            #target["Name Match"][i] = ratio_list[0][0]
            #target["Name Match Score"][i] = ratio_list[0][1]
            
            if ratio_list[0][1] > 0.66:
                target["Bad Actor Flag"][i] = 1
            
            
    return target

df_final = my_fuzzy(df_final, badActor)

count_bad = df_final[df_final["Bad Actor Flag"] == 1]
print "**********"
print "Total number of bad actors including via fuzzy logic", len(count_bad)

2018-10-17 13:26:37.729000
len before drop 1224
len after drop 1072
len after drop 965
number of bad actors 47
Start fuzzy matching
iteration 0
2018-10-17 13:26:37.797000
iteration 500
2018-10-17 13:26:37.994000
iteration 1000
2018-10-17 13:26:38.179000
**********
Total number of bad actors including via fuzzy logic 56


In [325]:
#df_final.to_csv(path + "test.csv", index=False)

PERCENT OF TRANSACTIONS WITH ONE STAFF MEMBER

In [326]:
industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

data_list = []

df2 = df.copy()

df2 = df2[["CUSTOMER NAME", "STAFF NAME"]]

df2["Count"] = 1

df2 = df2.groupby(['CUSTOMER NAME', 'STAFF NAME']).sum()
df2 = df2.add_suffix('').reset_index()

#for i in range(0, 1):
for i in range(0, len(industry_names)):
    
    if i%500 == 0:
        print i
    
    df3 = df2[df2["CUSTOMER NAME"] == industry_names[i]]
    df3 = df3.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    
    sum_trans = df3["Count"].sum()

    df3["p"] = df3["Count"]/float(sum_trans)
    df3["p"] = (df3["p"].round(3)) * 100
    
    perc_list = []
    
    if len(df3) == 1:
        for j in range(0, 3):
            if (j==1 or j==2):
                perc_list.append(['NA', 'NA'])
            else:
                perc_list.append([df3["STAFF NAME"][j], df3["p"][j] ])
    
    elif len(df3) == 2:
        for j in range(0, 3):
            if j == 2:
                perc_list.append(['NA', 'NA'])
            else:
                perc_list.append([df3["STAFF NAME"][j], df3["p"][j] ])
    
    elif len(df3) >= 3:
        for j in range(0, 3):
            perc_list.append([df3["STAFF NAME"][j], df3["p"][j] ])
    
    #df3 = df3.reset_index()
    #data_list.append([industry_names[i], perc_list[0][0], perc_list[0][1], perc_list[1][0], perc_list[1][1], perc_list[2][0], perc_list[2][1]  ])
    #data_list.append([industry_names[i], perc_list[0][0], perc_list[0][1], perc_list[1][0], perc_list[1][1]  ])
    data_list.append([industry_names[i], perc_list[0][0], perc_list[0][1]])
    
    #print "sum of trans", sum_trans
    #print perc_list
    #print df3
    
    #print len(df3)

    
#trans = pd.DataFrame(data_list, columns=('CUSTOMER NAME', 'High Staff Trans 1', 'Percent 1', 'High Staff Trans 2', 'Percent 2', 'High Staff Trans 3', 'Percent 3'))
#trans = pd.DataFrame(data_list, columns=('CUSTOMER NAME', 'High Staff Trans 1', 'Percent 1', 'High Staff Trans 2', 'Percent 2'))
trans = pd.DataFrame(data_list, columns=('CUSTOMER NAME', 'High Staff Trans', 'Percent'))

#MERGE WTIH INDUSTRY NAME DF
df_final = pd.merge(df_final, trans, how='left', on=['CUSTOMER NAME'])




0
500
1000


HIGH PERCENT OF TRANSACTION TYPE WITH ONE STAFF TRANS HIGHEST PERCENT 

In [327]:
del_serv = set(df["DELIVERED_SERVICE"].tolist() )

print len(del_serv)

df2 = df.copy()

#get only the names in df_final
industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

df2 = df2[df2['CUSTOMER NAME'].isin(industry_names)]
df2 = df2.reset_index(drop=True)


df2 = df2[["CUSTOMER NAME", "STAFF NAME", "DELIVERED_SERVICE"]]

df2["Count"] = 1
df2G = df2.groupby(['CUSTOMER NAME', 'STAFF NAME', 'DELIVERED_SERVICE']).sum()
df2G = df2G.add_suffix('').reset_index()


df_final["High Staff Trans Delivered Servc"] = ""
df_final["# Delivered Servc"] = ""
df_final["% Delivered Servc"] = ""

for i in range(0, len(df_final)):
    if (i%500) == 0:
        print i
    
    cname = df_final["CUSTOMER NAME"][i]
    hsname = df_final["High Staff Trans"][i]
    
     
    df3G = df2G[df2G['CUSTOMER NAME'].isin([cname])]
    df3G = df3G[df3G['STAFF NAME'].isin([hsname])]
    df3G = df3G.reset_index(drop=True)
    
    df3G = df3G.sort_values(by = 'Count', ascending=False).reset_index(drop=True)
    
    df_final["High Staff Trans Delivered Servc"][i] = df3G["DELIVERED_SERVICE"][0]
    df_final["# Delivered Servc"][i] = df3G["Count"][0]
    
    sum_ds = df3G["Count"].sum()
    
    df_final["% Delivered Servc"][i] = (df3G["Count"][0]/float(sum_ds) ) * 100
    
    
    #print df3G
    
df_final["% Delivered Servc"] = df_final["% Delivered Servc"].astype(float)
df_final =   df_final.round(1)  

11
0
500
1000


HIGH PERCENT OF TRANSACTIONS WITH ONE STAFF MEMBER MEETING MORE THAN 2 TIMES ON ONE DAY

In [328]:
df2 = df.copy()

#get only the names in df_final
industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

df2 = df2[df2['CUSTOMER NAME'].isin(industry_names)]
df2 = df2.reset_index(drop=True)

df2["DateTime"] = df2["YEAR"] + "-" + df2["MONTH"] + "-" + df2["DAY"] + " " + df2["HOUR"] + ":" + df2["MINUTE"]

df2["Date"] = df2["YEAR"] + "-" + df2["MONTH"] + "-" + df2["DAY"] 

df2["DateTime"] = pd.to_datetime(df2["DateTime"])

df2["Date"] = pd.to_datetime(df2["Date"])


df2 = df2.assign(session=pd.cut(df2.DateTime.dt.hour,[0,6,12,18,24],labels=['Night','Morning','Afternoon','Evening']))

session_dummies = pd.get_dummies(df2.session, prefix='session')
print "Length of session dummies", len(session_dummies)
print "Length of df2", len(df2)
print "Difference", len(df2) - len(session_dummies)
df2 = pd.concat([df2, session_dummies], axis=1)
#df2 = df2.drop('JobType', 1)

df2 = df2.drop('session_Night', 1)
df2 = df2.drop('session_Evening', 1)

df2 = df2[["CUSTOMER NAME", "STAFF NAME", "Date", "session_Morning", "session_Afternoon"]]

df2G = df2.groupby(['CUSTOMER NAME', 'STAFF NAME', 'Date']).sum()

df2G = df2G.add_suffix('').reset_index()

df2G["Total Day"] = df2G["session_Morning"] + df2G["session_Afternoon"]


#Merge df_final with df2G
df_final["High Staff Day Trans"] = ""
df_final["Total Day"] = ""
df_final["session_Morning"] = ""
df_final["session_Afternoon"] = ""



for i in range(0, len(df_final)):
#for i in range(0, 30):
    
    if (i%500) == 0:
        print i
    
    cname = df_final["CUSTOMER NAME"][i]
    #hsname = df_final["High Staff Trans"][i]
    
    df3G = df2G[df2G['CUSTOMER NAME'].isin([cname])]
    #df3G = df3G[df3G['STAFF NAME'].isin([hsname])]
    df3G = df3G.reset_index(drop=True)
    
    df3G = df3G.sort_values(by = 'Total Day', ascending=False).reset_index(drop=False)


    df_final["High Staff Day Trans"][i] = df3G["STAFF NAME"][0]
    df_final["Total Day"][i] = df3G["Total Day"][0]
    df_final["session_Morning"][i] = df3G["session_Morning"][0]
    df_final["session_Afternoon"][i] = df3G["session_Afternoon"][0]
    


Length of session dummies 41540
Length of df2 41540
Difference 0
0
500
1000


In [329]:
#df_final.to_csv(path + "test.csv", index=False)

In [330]:
#df.to_csv(path + "test.csv", index=False)

In [331]:
#qn.to_csv(path + "customer_names.csv", index=False)

# PART 2

Industry member with more than two cerfificat of corrections and drop offs in one day

In [332]:
df2 = df.copy()

#print "Delivered Service "
#del_service = set(df["DELIVERED_SERVICE"].tolist() )
#print del_service

industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

df2 = df2[df2['CUSTOMER NAME'].isin(industry_names)]
df2 = df2.reset_index(drop=True)

df2["Date"] = df2["YEAR"] + "-" + df2["MONTH"] + "-" + df2["DAY"] 

df2["Date"] = pd.to_datetime(df2["Date"])

df2["Date"]  = df2["Date"].apply(lambda x: x.date())

df2 = df2[["CUSTOMER NAME", "DELIVERED_SERVICE", "Date"]]

service_dummies = pd.get_dummies(df2.DELIVERED_SERVICE, prefix='Delivered_Service')
print "Length of service_dummies dummies", len(service_dummies)
print "Length of df2", len(df2)
print "Difference", len(df2) - len(service_dummies)
df2 = pd.concat([df2, service_dummies], axis=1)

#aggregate drop off and certificate of corrections in person
df2["Drop-off & Cert Corr"] = df2["Delivered_Service_DROP OFF"] + df2["Delivered_Service_DROP OFF (10X10)"] + df2["Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)"] + df2["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"]

df2 = df2.drop('Delivered_Service_DROP OFF', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (10X10)', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)', 1)
df2 = df2.drop('Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)', 1)


#df2 = df2[["CUSTOMER NAME","Date", "Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)", "Drop-off All"]]
df2 = df2[["CUSTOMER NAME","Date", "Drop-off & Cert Corr"]]

df2G = df2.groupby(['CUSTOMER NAME',  'Date']).sum()

df2G = df2G.add_suffix('').reset_index()

#df_final[">2 Cert Corr Flag"] = "No"
#df_final[">2 Cert Corr Dates"]  = ""

#df_final[">2 Drop Off Flag"] = "No"
#df_final[">2 Drop Off Dates"]  = ""

df_final[">2 COC & DO Flag"] = "No"
df_final[">2 COC & DO Dates"]  = ""



for i in range(0, len(df_final)):
#for i in range(0, 3):
    
    if (i%500) == 0:
        print i
    
    cname = df_final["CUSTOMER NAME"][i]
    
    df3G = df2G[df2G['CUSTOMER NAME'].isin([cname])]
    df3G = df3G.reset_index(drop=True)
    
    #df3G = df3G.sort_values(by = 'Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)', ascending=False).reset_index(drop=False)
    
    #df4G = df3G[df3G["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"] > 2]
    
    df4G = df3G[df3G["Drop-off & Cert Corr"] > 2]
    
    if len(df4G) > 0:
        df_final[">2 COC & DO Flag"][i] = "Yes"
        date_list = df4G["Date"].tolist()
        
        #convert date to string
        date_list2 = []
        for j in range(0, len(date_list)):
            date_list2.append(date_list[j].strftime('%m/%d/%Y'))
            #print date_list[j].strftime('%m/%d/%Y')
        
        df_final[">2 COC & DO Dates"][i] =  date_list2
        
    #df4G = df3G[df3G["Drop-off All"] > 2]
    
    #if len(df4G) > 0:
    #    df_final[">2 Drop Off Flag"][i] = "Yes"
    #    date_list = df4G["Date"].tolist()
    #    df_final[">2 Drop Off Dates"][i] =  date_list
        

    #print df3G

Length of service_dummies dummies 41540
Length of df2 41540
Difference 0
0
500
1000


In [333]:
#df_final.to_csv(path + "test.csv", index=False)

In [334]:
#df2G.to_csv(path + "test.csv", index=False)

HIGH PERCENTAGE OF CERTIFICATE OF CORRECTIONS (IN PERSON) AND DROP OFF TRANSACTIONS

In [335]:
df2 = df.copy()

#print "Delivered Service "
#del_service = set(df["DELIVERED_SERVICE"].tolist() )
#print del_service

industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

df2 = df2[df2['CUSTOMER NAME'].isin(industry_names)]
df2 = df2.reset_index(drop=True)

df2 = df2[["CUSTOMER NAME", "DELIVERED_SERVICE"]]

service_dummies = pd.get_dummies(df2.DELIVERED_SERVICE, prefix='Delivered_Service')
print "Length of service_dummies dummies", len(service_dummies)
print "Length of df2", len(df2)
print "Difference", len(df2) - len(service_dummies)
df2 = pd.concat([df2, service_dummies], axis=1)

#df2 = df2[["CUSTOMER NAME","Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)", "Delivered_Service_DROP OFF"]]

#aggregate drop off and certificate of corrections in person
df2["Drop-off & Cert Corr"] = df2["Delivered_Service_DROP OFF"] + df2["Delivered_Service_DROP OFF (10X10)"] + df2["Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)"] + df2["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"]

df2 = df2.drop('Delivered_Service_DROP OFF', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (10X10)', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)', 1)
df2 = df2.drop('Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)', 1)

df2G = df2.groupby(['CUSTOMER NAME']).sum()
df2G = df2G.add_suffix('').reset_index()



#Add all of the rows of data
col_names = sorted(list(df2G.columns.values))
df2G["Total"] = 0
#strip all columns of white space and convert to uppper
for i in range(1, len(col_names)):
    df2G["Total"] = df2G["Total"] + df2G[col_names[i]]
    
    
#df2G["%Certificat of Corr in-Person"] = (df2G["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"]/df2G["Total"]) *100
#df2G["%Drop-off"] = (df2G["Drop-off All"]/df2G["Total"]) *100

df2G["%COC & DO"] = (df2G["Drop-off & Cert Corr"]/df2G["Total"]) *100


df2G = df2G.round(1)
df2G = df2G[["CUSTOMER NAME",  "%COC & DO"]]

df_final = pd.merge(df_final, df2G, how='left', on=['CUSTOMER NAME'])



Length of service_dummies dummies 41540
Length of df2 41540
Difference 0


HIGH PERCENTAGE OF CERTIFICATE OF CORRECTIONS AND DROP OFF WITH ONE STAFF MEMBER

In [336]:
df2 = df.copy()

industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

df2 = df2[df2['CUSTOMER NAME'].isin(industry_names)]
df2 = df2.reset_index(drop=True)

df2 = df2[["CUSTOMER NAME","STAFF NAME", "DELIVERED_SERVICE"]]

service_dummies = pd.get_dummies(df2.DELIVERED_SERVICE, prefix='Delivered_Service')
print "Length of service_dummies dummies", len(service_dummies)
print "Length of df2", len(df2)
print "Difference", len(df2) - len(service_dummies)
df2 = pd.concat([df2, service_dummies], axis=1)


#aggregate drop off and certificate of corrections in person
df2["Drop-off & Cert Corr"] = df2["Delivered_Service_DROP OFF"] + df2["Delivered_Service_DROP OFF (10X10)"] + df2["Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)"] + df2["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"]

df2 = df2.drop('Delivered_Service_DROP OFF', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (10X10)', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)', 1)
df2 = df2.drop('Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)', 1)

df2G = df2.groupby(['CUSTOMER NAME', "STAFF NAME"]).sum()
df2G = df2G.add_suffix('').reset_index()

#Add all of the rows of data
col_names = sorted(list(df2G.columns.values))
df2G["Total"] = 0
#strip all columns of white space and convert to uppper
for i in range(2, len(col_names)):
    if df2G[col_names[i]].dtypes != 'O':
        df2G["Total"] = df2G["Total"] + df2G[col_names[i]]
    
    
#df2G["%Certificat of Corr in-Person"] = (df2G["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"]/df2G["Total"]) *100
#df2G["%Drop-off"] = (df2G["Drop-off All"]/df2G["Total"]) *100
#df2G = df2G.round(1)
df2G = df2G[["CUSTOMER NAME", "STAFF NAME",  "Drop-off & Cert Corr"]]


df_final["High % COC & DO Staff"] = ""
df_final["% COC & DO Staff"] = ""

#df_final["High % Drop-off Staff"] = ""
#df_final["% Drop-off Staff"] = ""


for i in range(0, len(df_final)):
#for i in range(0, 1):
    if (i%500) == 0:
        print i
    
    cname = df_final["CUSTOMER NAME"][i]

     
    df3G = df2G[df2G['CUSTOMER NAME'].isin([cname])]
    df3G = df3G.reset_index(drop=True)
    df3G = df3G.sort_values(by = 'Drop-off & Cert Corr', ascending=False).reset_index(drop=True)
    
    sum_cert = df3G["Drop-off & Cert Corr"].sum()
    

    
    #print sum_cert
    
    df_final["High % COC & DO Staff"][i] = df3G["STAFF NAME"][0]
    df_final["% COC & DO Staff"][i] = ( (df3G["Drop-off & Cert Corr"][0]/float(sum_cert) ).round(3) ) *100
 
    #Drop-offs
    #sum_drop = df3G["Drop-off All"].sum()
    #df3G = df3G.sort_values(by = 'Drop-off All', ascending=False).reset_index(drop=True)

    #df_final["High % Drop-off Staff"][i] = df3G["STAFF NAME"][0]
    #df_final["% Drop-off Staff"][i] = ( (df3G["Drop-off All"][0]/float(sum_drop) ).round(3) ) *100
 

        
    #print
    #print df3G
   
    
    


Length of service_dummies dummies 41540
Length of df2 41540
Difference 0
0
500


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: RuntimeWarning: invalid value encountered in divide
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: RuntimeWarning: invalid value encountered in rint


1000


HIGH PERCENTAGE OF TRANSACTIONS IN A SINGLE DAY WITH ONE STAFF MEMBER

In [337]:
df2 = df.copy()

#get only the names in df_final
industry_names = sorted(list(set(df_final["CUSTOMER NAME"].tolist())))

df2 = df2[df2['CUSTOMER NAME'].isin(industry_names)]
df2 = df2.reset_index(drop=True)

df2["DateTime"] = df2["YEAR"] + "-" + df2["MONTH"] + "-" + df2["DAY"] + " " + df2["HOUR"] + ":" + df2["MINUTE"]

df2["Date"] = df2["YEAR"] + "-" + df2["MONTH"] + "-" + df2["DAY"] 

df2["DateTime"] = pd.to_datetime(df2["DateTime"])

df2["Date"] = pd.to_datetime(df2["Date"])


df2 = df2.assign(session=pd.cut(df2.DateTime.dt.hour,[0,6,12,18,24],labels=['Night','Morning','Afternoon','Evening']))

session_dummies = pd.get_dummies(df2.session, prefix='session')
print "Length of session dummies", len(session_dummies)
print "Length of df2", len(df2)
print "Difference", len(df2) - len(session_dummies)
df2 = pd.concat([df2, session_dummies], axis=1)
#df2 = df2.drop('JobType', 1)

df2 = df2.drop('session_Night', 1)
df2 = df2.drop('session_Evening', 1)



df2 = df2[["CUSTOMER NAME", "STAFF NAME","DELIVERED_SERVICE", "Date", "session_Morning", "session_Afternoon"]]

#create service dummies
service_dummies = pd.get_dummies(df2.DELIVERED_SERVICE, prefix='Delivered_Service')
print "Length of service_dummies dummies", len(service_dummies)
print "Length of df2", len(df2)
print "Difference", len(df2) - len(service_dummies)
df2 = pd.concat([df2, service_dummies], axis=1)

#aggregate drop off and certificate of corrections in person
df2["Drop-off & Cert Corr"] = df2["Delivered_Service_DROP OFF"] + df2["Delivered_Service_DROP OFF (10X10)"] + df2["Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)"] + df2["Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)"]

df2 = df2.drop('Delivered_Service_DROP OFF', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (10X10)', 1)
df2 = df2.drop('Delivered_Service_DROP OFF (MULTIPLE SUBMISSION)', 1)
df2 = df2.drop('Delivered_Service_CERTIFICATE OF CORRECTION (IN-PERSON)', 1)

df2 = df2[["CUSTOMER NAME", "STAFF NAME","Date", "session_Morning", "session_Afternoon", "Drop-off & Cert Corr"]]

df2 = df2.rename(columns={'Drop-off & Cert Corr': '#COC & DO'})


df2G = df2.groupby(['CUSTOMER NAME', 'STAFF NAME', 'Date']).sum()
df2G = df2G.add_suffix('').reset_index()

df2G = df2G[df2G["#COC & DO"] > 0].reset_index(drop=True)

#df2G["Total Day"] = df2G["session_Morning"] + df2G["session_Afternoon"]


Length of session dummies 41540
Length of df2 41540
Difference 0
Length of service_dummies dummies 41540
Length of df2 41540
Difference 0


EXPORT HIGH PERCENT OF TRANSACTION IN A SINGLE DAY, DIFF TIMES, WITH ONE STAFF MEMBER

In [54]:
df2G.to_csv("C:\\Users\\dmehri\\Documents\\DATA\\AEU Network Analysis\\High Perc Trans Single Day Diff Time w Staff Member.csv", index=False)

EXPORT NETWORK ANALYSIS

In [373]:
df_final.to_csv(path + "AEU Network Analysis.csv", index=False)

# PART 3: OUTLIER ANALYSIS

### Z-score calculations

Normal distributed variables

In [338]:
import numpy as np

df_final["Percent"] = df_final["Percent"].astype(float)
df_final["Percent Z-score"] = (df_final["Percent"] - df_final["Percent"].mean() )/df_final["Percent"].std()

df_final["% COC & DO Staff"] = df_final["% COC & DO Staff"].astype(float)
df_final["% COC & DO Staff Z-score"] = (df_final["% COC & DO Staff"] - df_final["% COC & DO Staff"].mean() )/df_final["% COC & DO Staff"].std()


#take natural log of morning and afternoon sessions and take z-score
#df_final["session_Morning"] = df_final["session_Morning"].astype(str)
#df_final["session_Afternoon"] = df_final["session_Afternoon"].astype(str)

#df_final["session_Morning"] = df_final["session_Morning"].str.replace('0', '')
#df_final["session_Afternoon"] = df_final["session_Afternoon"].str.replace('0', '')

df_final["session_Morning"] = df_final["session_Morning"].astype(float)
df_final["session_Afternoon"] = df_final["session_Afternoon"].astype(float)


std = df_final[df_final["session_Morning"] != 0.0]
std = df_final["session_Morning"].std()
mean = df_final[df_final["session_Morning"] != 0.0]
mean = df_final["session_Morning"].mean()

print "std"
print std
print "mean"
print mean

df_final["session_Morning"] = df_final["session_Morning"].replace(0, np.nan)
#df_final["LN session_Morning"] = np.log(df_final["session_Morning"])
df_final["session_Morning Z-score"] = (df_final["session_Morning"] - mean )/std


std = df_final[df_final["session_Afternoon"] != 0]
std = df_final["session_Afternoon"].std()
mean = df_final[df_final["session_Afternoon"] != 0]
mean = df_final["session_Afternoon"].mean()

print 
print std
print mean

df_final["session_Afternoon"] = df_final["session_Afternoon"].replace(0, np.nan)
#df_final["LN session_Afternoon"] = np.log(df_final["session_Afternoon"])
df_final["session_Afternoon Z-score"] = (df_final["session_Afternoon"] - mean )/std



std
1.72132549975
mean
1.46993780235

1.84127739433
1.08154803041


### Z-Score

In [377]:

zscore = df_final[[ "CUSTOMER NAME","Percent Z-score", "% COC & DO Staff Z-score", "session_Morning Z-score", "session_Afternoon Z-score"]]

#zscore

#zscore["Z-score Avg"] = ""

zscore.index = zscore["CUSTOMER NAME"]

zscore = zscore.drop('CUSTOMER NAME', 1)

zscore['Z-score Avg'] = zscore.mean(axis=1)

zscore = zscore.sort_values(by = 'Z-score Avg', ascending=False).reset_index(drop=False)

zscore = zscore.round(2)

#zscore = zscore.reset_index(drop=False)

#for i in range(0, len(zscore)):
#    if zscore["session_Afternoon Z-score"][i] is null:
#        print "nan"


C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\series.py:1342: RuntimeWarning: invalid value encountered in rint
  result = _values_from_object(self).round(decimals)


In [378]:
zscore.to_csv(path + "Outlier Analysis.csv", index=False)

In [341]:
df_final.dtypes

CUSTOMER NAME                        object
Transaction Count                   float64
Daly Transaction                      int32
Livingston Transaction                int32
Total Supervisor Transaction          int32
% Daly                              float64
% Livingston                        float64
Bad Actor Flag                        int64
High Staff Trans                     object
Percent                             float64
High Staff Trans Delivered Servc     object
# Delivered Servc                    object
% Delivered Servc                   float64
High Staff Day Trans                 object
Total Day                            object
session_Morning                     float64
session_Afternoon                   float64
>2 COC & DO Flag                     object
>2 COC & DO Dates                    object
%COC & DO                           float64
High % COC & DO Staff                object
% COC & DO Staff                    float64
Percent Z-score                 

Modified Z-score for non normal distributions

In [312]:
"""

#M_z-score = 0.6745 * (xi - xmedian)/median(abs(xi - xmedian))
#MAD = median(abs(xi - xmedian))

#df_final["session_Morning"] = df_final["session_Morning"].astype(int)
df_final["session_Morning"] = df_final["session_Morning"].replace(0, np.nan)
#df_final["session_Afternoon"] = df_final["session_Afternoon"].astype(int)
df_final["session_Afternoon"] = df_final["session_Afternoon"].replace(0, np.nan)

df_final["Total Day"] = df_final["Total Day"].astype(int)



#Morning
#median = df_final[df_final["session_Morning"] != 0]
median = df_final["session_Morning"].median()

print median

df_final["session_Morning Modified Z-score"] =  0.6745 * (df_final["session_Morning"] - median ) 

df_final["MAD"] = df_final["session_Morning"] - median
#df_final["MAD"] = df_final["MAD"].abs()
#df_final["MAD"] = df_final["MAD"].median()

#df_final["session_Morning Modified Z-score"] = df_final["session_Morning Modified Z-score"] / df_final["MAD"]  


#Afternoon
#df_final["session_Afternoon Modified Z-score"] =  0.6745 * (df_final["session_Afternoon"] - df_final["session_Afternoon"].median() ) 

#df_final["MAD"] = df_final["session_Afternoon"] - df_final["session_Afternoon"].median()
#df_final["MAD"] = df_final["MAD"].abs()
#df_final["MAD"] = df_final["MAD"].median()

#df_final["session_Afternoon Modified Z-score"] = df_final["session_Afternoon Modified Z-score"] / df_final["MAD"]  


"""

2.0


In [141]:
print "total day"
total = df_final["Total Day"].sum()
print total

print "total morning"
total = df_final["session_Morning"].sum()
print total

print "total afternoon"
total = df_final["session_Afternoon"].sum()
print total

print "afternoon median"
median = df_final["session_Afternoon"].median()
print median
print
print "len of df final"
print len(df_final)

print 
df_final["Total Day"] = df_final["Total Day"].astype(int)
total = df_final["Total Day"].sum()
print total

print 
total = 0

print

total = 0

for i in range(0, len(df_final)):
    #print df_final["Total Day"][i]
    
    total = total + df_final["Total Day"][i]
    
print total

total day
3692
total morning
2127
total afternoon
1565
afternoon median
0.0

len of df final
1447

3692


3692
